# Modelling

In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('../')

import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [ ]:
# Load final dataset
df = pd.read_csv('../data/processed/features/final_dataset.csv')